In [11]:
import re
import MeCab
import pandas as pd

# **教育漢字リスト（文部科学省の1,026字）**
education_kanji = set("""
一右雨円王音下火花貝学気九休玉金空月犬見五口校左三山子四糸字耳七車手十出女小上森人水正生青夕石赤千川先早草足村大男竹中虫町天田土二日入年白八百文木本名目立力林六
引羽雲園遠何科夏家歌画回会海絵外角楽活間丸岩顔汽記帰弓牛魚京強教近兄形計元言原戸古午後語工公広交光考行高黄合谷国黒今才細作算止市矢姉思紙寺自時室社弱首秋週春
書少場色食心新親図数西声星晴切雪船線前組走多太体台地池知茶昼長鳥朝直通弟店点電刀冬当東答頭同道読内南肉馬売買麦半番父風分聞米歩母方北毎妹万明鳴毛門夜野友用曜来
里理話悪安暗医委意育員院飲運泳駅央横屋温化荷界開階寒感漢館岸起期客究急級宮球去橋業曲局銀区苦具君係軽血決研県庫湖向幸港号根祭皿仕死使始指歯詩次事持式実写者主
守取酒受州拾終習集住重宿所暑助昭消商章勝乗植申身神真深進世整昔全相送想息速族他打対待代第題炭短談着注柱丁帳調追定庭笛鉄転都度投豆島湯登等動童農波配倍箱畑発反坂
板皮悲美鼻筆氷表秒病品負部服福物平返勉放味命面問役薬由油有遊予羊洋葉陽様落流旅両緑礼列練路和愛案以衣位囲胃印英栄塩億加果貨課芽改械害街各覚完官管関観願希季紀
""".replace("\n", ""))

# **ファイルの読み込み**
#file_path = "ir/2025/40610_デンカ/denkareport2024.txt"
file_path = "ir/2025/40040_レゾナック・ホールディングス/pdf-sustainability-report-integratedreport-RESONAC24J_spread.txt"

with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

text = "これは、むずかしいかんじがどのぐらいあるかをしらべるための123もじからなるぶんしょうです。"

# **MeCabによる形態素解析**
mecab = MeCab.Tagger("-Owakati")
words = mecab.parse(text).strip().split()

# **文章数（句点「。」でカウント）**
num_sentences = text.count("。")

# **文字数と単語数**
total_chars = len(text.replace("\n", ""))
num_words = len(words)

# **1文あたりの単語数**
avg_sentence_length = num_words / num_sentences if num_sentences else 0

# **ひらがな・カタカナ・漢字の比率**
hiragana_count = sum(1 for char in text if 'ぁ' <= char <= 'ん')
katakana_count = sum(1 for char in text if 'ァ' <= char <= 'ン' or char == 'ー')
kanji_count = sum(1 for char in text if '一' <= char <= '龥')

hiragana_ratio = (hiragana_count / total_chars) * 100
katakana_ratio = (katakana_count / total_chars) * 100
kanji_ratio = (kanji_count / total_chars) * 100

# **修正フォグ指数**
long_kanji_words = sum(1 for word in words if re.match(r'^[一-龥]{2,}$', word))
long_kanji_ratio = (long_kanji_words / num_words) * 100 if num_words else 0
fog_index = 0.4 * (avg_sentence_length + 100 * long_kanji_ratio)

# **Coleman-Liau Index（修正）**
L = (total_chars / num_words) * 100 if num_words else 0
S = (num_sentences / num_words) * 100 if num_words else 0
coleman_liau = 0.0588 * L - 0.296 * S - 15.8

# **難読語の判定**
def is_difficult_word(word):
    """ 難読語の判定ルール """
    if re.match(r'^[ぁ-んァ-ンーa-zA-Z0-9]+$', word):  # ひらがな・カタカナ・英数字のみは除外
        return False
    return any(char not in education_kanji for char in word)

difficult_words = {word for word in words if is_difficult_word(word)}
print(difficult_words)
difficult_word_ratio = (len(difficult_words) / num_words) * 100 if num_words else 0

# **データフレームにまとめる**
df = pd.DataFrame([
    ["総文字数", total_chars],
    ["総単語数", num_words],
    ["総文数", num_sentences],
    ["1文あたりの単語数", avg_sentence_length],
    ["ひらがな比率 (%)", hiragana_ratio],
    ["カタカナ比率 (%)", katakana_ratio],
    ["漢字比率 (%)", kanji_ratio],
    ["修正フォグ指数", fog_index],
    ["Coleman-Liau Index（修正版）", coleman_liau],
    ["難読語の比率 (%)", difficult_word_ratio]
], columns=["指標", "値"])

# **結果を表示**
print(df)


{'、', '。'}
                        指標          値
0                     総文字数  46.000000
1                     総単語数  21.000000
2                      総文数   1.000000
3                1文あたりの単語数  21.000000
4               ひらがな比率 (%)  89.130435
5               カタカナ比率 (%)   0.000000
6                 漢字比率 (%)   0.000000
7                  修正フォグ指数   8.400000
8  Coleman-Liau Index（修正版）  -4.329524
9               難読語の比率 (%)   9.523810


In [25]:
import re
import MeCab
import pandas as pd

# **教育漢字リスト（文部科学省の1,026字）**
education_kanji = set("""
一右雨円王音下火花貝学気九休玉金空月犬見五口校左三山子四糸字耳七車手十出女小上森人水正生青夕石赤千川先早草足村大男竹中虫町天田土二日入年白八百文木本名目立力林六
引羽雲園遠何科夏家歌画回会海絵外角楽活間丸岩顔汽記帰弓牛魚京強教近兄形計元言原戸古午後語工公広交光考行高黄合谷国黒今才細作算止市矢姉思紙寺自時室社弱首秋週春
書少場色食心新親図数西声星晴切雪船線前組走多太体台地池知茶昼長鳥朝直通弟店点電刀冬当東答頭同道読内南肉馬売買麦半番父風分聞米歩母方北毎妹万明鳴毛門夜野友用曜来
里理話悪安暗医委意育員院飲運泳駅央横屋温化荷界開階寒感漢館岸起期客究急級宮球去橋業曲局銀区苦具君係軽血決研県庫湖向幸港号根祭皿仕死使始指歯詩次事持式実写者主
守取酒受州拾終習集住重宿所暑助昭消商章勝乗植申身神真深進世整昔全相送想息速族他打対待代第題炭短談着注柱丁帳調追定庭笛鉄転都度投豆島湯登等動童農波配倍箱畑発反坂
板皮悲美鼻筆氷表秒病品負部服福物平返勉放味命面問役薬由油有遊予羊洋葉陽様落流旅両緑礼列練路和愛案以衣位囲胃印英栄塩億加果貨課芽改械害街各覚完官管関観願希季紀
喜旗器機議求泣救給挙漁共協鏡競極訓軍郡径型景芸欠結建健験固功好候航康告差菜最材昨札刷殺察参産散残士氏史司試児治辞失借種周祝順初松笑唱焼象照賞臣信成省清静席積折
節説浅戦選然争倉巣束側続卒孫帯隊達単置仲貯兆腸低底停的典伝徒努灯堂働特得毒熱念敗梅博飯飛費必票標不夫付府副粉兵別辺変便包法望牧末満未脈民無約勇要養浴利陸良料量
輪類令冷例歴連老労録圧移因永営衛易益液演応往桜恩可仮価河過賀快解格確額刊幹慣眼基寄規技義逆久旧居許境均禁句群経潔件券険検限現減故個護効厚耕鉱構興講混査再災妻
採際在財罪雑酸賛支志枝師資飼示似識質舎謝授修述術準序招承証条状常情織職制性政勢精製税責績接設舌絶銭祖素総造像増則測属率損退貸態団断築張提程適敵統銅導徳独任燃
能破犯判版比肥非備俵評貧布婦富武復複仏編弁保墓報豊防貿暴務夢迷綿輸余預容略留領異遺域宇映延沿我灰拡革閣割株干巻看簡危机揮貴疑吸供胸郷勤筋系敬警劇激穴絹権憲源
厳己呼誤后孝皇紅降鋼刻穀骨困砂座済裁策冊蚕至私姿視詞誌磁射捨尺若樹収宗就衆従縦縮熟純処署諸除将傷障城蒸針仁垂推寸盛聖誠宣専泉洗染善奏窓創装層操蔵臓存尊宅担
探誕段暖値宙忠著庁頂潮賃痛展討党糖届難乳認納脳派拝背肺俳班晩否批秘腹奮並陛閉片補暮宝訪亡忘棒枚幕密盟模訳郵優幼欲翌乱卵覧裏律臨朗論
""".replace("\n", ""))

# **除外すべき記号・句読点リスト**
excluded_chars = set("。、！？（）【】『』「」・…＠＆％＃＊＋＝｜<>[]{}")

# **ファイルの読み込み**
#file_path = "ir/2025/40610_デンカ/denkareport2024.txt"
file_path = "ir/2025/40040_レゾナック・ホールディングス/pdf-sustainability-report-integratedreport-RESONAC24J_spread.txt"

with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

text = "見出し項目\n漢字を確認する目的の文章。カタカナもバリ100％include。単語の範囲は明々白々。"


# **MeCabによる形態素解析**
mecab = MeCab.Tagger("-Owakati")
words = mecab.parse(text).strip().split()
print(words)

# **文章数（句点「。」または改行「\n」でカウント）**
sentences = re.split(r'[。\n]+', text.strip())  # 句点「。」または改行「\n」で分割
print(sentences)
num_sentences = len(sentences)

# **文字数と単語数**
total_chars = len(text.replace("\n", ""))
num_words = len(words)

# **1文あたりの単語数**
avg_sentence_length = num_words / num_sentences if num_sentences else 0

# **ひらがな・カタカナ・漢字の比率**
hiragana_count = sum(1 for char in text if 'ぁ' <= char <= 'ん')
katakana_count = sum(1 for char in text if 'ァ' <= char <= 'ヶ' or char == 'ー')
kanji_count = sum(1 for char in text if '一' <= char <= '龥' or char == '々')

hiragana_ratio = (hiragana_count / total_chars) * 100
katakana_ratio = (katakana_count / total_chars) * 100
kanji_ratio = (kanji_count / total_chars) * 100

# **修正フォグ指数**
long_kanji_words = sum(1 for word in words if re.match(r'^[一-龥]{2,}$', word))
long_kanji_ratio = (long_kanji_words / num_words) * 100 if num_words else 0
fog_index = 0.4 * (avg_sentence_length + 100 * long_kanji_ratio)

# **Coleman-Liau Index（修正）**
L = (total_chars / num_words) * 100 if num_words else 0
S = (num_sentences / num_words) * 100 if num_words else 0
coleman_liau = 0.0588 * L - 0.296 * S - 15.8

# **難読語の判定**
def is_difficult_word(word):
    """ 難読語の判定ルール """
    if re.match(r'^[ぁ-んァ-ヶーa-zA-Z0-9]+$', word):  # ひらがな・カタカナ・英数字のみは除外
        return False
    if all(char in excluded_chars for char in word):  # 記号のみの単語を除外
        return False

    # ひらがな・カタカナを除去した文字列を取得
    filtered_word = re.sub(r'[ぁ-んァ-ヶー]', '', word)

    # 残った漢字がすべて教育漢字であれば難読語ではない
    return any(char not in education_kanji for char in filtered_word)

difficult_words = {word for word in words if is_difficult_word(word)}
print(difficult_words)
difficult_word_ratio = (len(difficult_words) / num_words) * 100 if num_words else 0

# **データフレームにまとめる**
df = pd.DataFrame([
    ["総文字数", total_chars],
    ["総単語数", num_words],
    ["総文数", num_sentences],
    ["1文あたりの単語数", avg_sentence_length],
    ["ひらがな比率 (%)", hiragana_ratio],
    ["カタカナ比率 (%)", katakana_ratio],
    ["漢字比率 (%)", kanji_ratio],
    ["修正フォグ指数", fog_index],
    ["Coleman-Liau Index（修正版）", coleman_liau],
    ["難読語の比率 (%)", difficult_word_ratio]
], columns=["指標", "値"])

# **結果を表示**
print(df)


['見出し', '項目', '漢字', 'を', '確認', 'する', '目的', 'の', '文章', '。', 'カタカナ', 'も', 'バリ', '100', '％', 'include', '。', '単語', 'の', '範囲', 'は', '明々', '白々', '。']
['見出し項目', '漢字を確認する目的の文章', 'カタカナもバリ100％include', '単語の範囲は明々白々', '']
{'範囲', '白々', '項目', '明々'}
                        指標            値
0                     総文字数    48.000000
1                     総単語数    24.000000
2                      総文数     5.000000
3                1文あたりの単語数     4.800000
4               ひらがな比率 (%)    16.666667
5               カタカナ比率 (%)    12.500000
6                 漢字比率 (%)    41.666667
7                  修正フォグ指数  1168.586667
8  Coleman-Liau Index（修正版）   -10.206667
9               難読語の比率 (%)    16.666667
